In [40]:
# rmbranto 2021/08 - original
# rmbranto 2021/10/12 - rewrite

# Demonstrate R programming lanquage 'spocc', 'scrubr' and 'ggplot2' packages to retrieve and combine
# selected invasive species occurrence data from multiple source, see also: 
# https://ocean.si.edu/ocean-life/5-invasive-species-you-should-know 
# https://docs.ropensci.org/spocc/

options(stringsAsFactors = FALSE)

library(rgdal)
library(spocc)
library(sf)
library(leaflet)
library(wellknown)
library(mregions)
library(rmapshaper)
library(scrubr)
#library(plyr)
#library(ggplot2)

########################################################################################
########################################################################################
# setup extraction run ...

fName<-'invasives'

species.list<-read.csv(paste(fName,'_list.csv',sep=''))
prov.list=read.csv('prov_list.csv')
pNames=prov.list$prov

# read shapefile

In [2]:
fao.loc="geobase/World_Fao_Zones.shp"
eez.loc="geobase/EEZ_Land_v3_202030.shp"
nocoast.loc="geobase/FAO_AREAS_CWP_NOCOASTLINE.shp"

fao.shp=sf::st_read(fao.loc,quiet=TRUE)
eez.shp=sf::st_read(eez.loc,quiet=TRUE)

eez.shp=eez.shp[eez.shp$POL_TYPE=='Union EEZ and country',]
eez.shp$ISO_TER1[is.na(eez.shp$ISO_TER1)]=eez.shp$MRGID_EEZ[is.na(eez.shp$ISO_TER1)]

In [59]:
eez.simple=rmapshaper::ms_simplify(eez.shp,keep=.05)
fao.simple=rmapshaper::ms_simplify(fao.shp,keep=.05)
eez.df=data.frame(eez.shp)

In [ ]:
nocoast.shp<- sf::st_read(nocoast.loc) 
nocoast.shp=nocoast.shp[nocoast.shp$F_LEVEL == 'MAJOR','F_CODE']
nocoast.shp$zone=nocoast.shp$F_CODE

In [61]:
leaflet()%>%
addProviderTiles(
    provider = 'Esri.OceanBasemap') %>%
addPolygons(data=nocoast.shp[nocoast.shp$zone%in%c(21),])

HTML widgets cannot be represented in plain text (need html)

In [54]:
#fao.simple=nocoast.shp[,'zone']
#fao.simple

zone geometry                      
1  18   MULTIPOLYGON (((-40 89.99, ...
2  61   MULTIPOLYGON (((180 20, 179...
3  67   MULTIPOLYGON (((-168.9119 6...
4  37   MULTIPOLYGON (((38 32, 38 3...
5  27   MULTIPOLYGON (((-12.7309 36...
6  31   MULTIPOLYGON (((-40 15, -40...
7  34   MULTIPOLYGON (((12.21097 -3...
8  51   MULTIPOLYGON (((80 -10, 80 ...
9  57   MULTIPOLYGON (((129 -25, 13...
10 71   MULTIPOLYGON (((-180 -25, -...
11 87   MULTIPOLYGON (((-67.27 -57....
12 41   MULTIPOLYGON (((-50 -53.2, ...
13 81   MULTIPOLYGON (((-180 -60, -...
14 58   MULTIPOLYGON (((86 -55, 86....
15 48   MULTIPOLYGON (((-88.71042 -...
16 88   MULTIPOLYGON (((150 -88, 15...
17 77   MULTIPOLYGON (((-100 31.214...
18 47   MULTIPOLYGON (((4.908014 -5...
19 21   MULTIPOLYGON (((-70 35, -70...

In [38]:
eez.TER=function(my.eez='',as.SOV=TRUE){
    if(my.eez==''){return('')}
    if(as.SOV){
        eez.df$ISO_TER1[eez.df$ISO_SOV1==my.eez]
    }else{
        eez.df$ISO_TER1[eez.df$ISO_TER1==my.eez]
    }
}

In [39]:
set_bbox=function(
    my.fao='',
    my.eez='',
    my.buff=1.0){
    
if(paste(my.eez,collapse='')!='' & paste(my.fao,collapse='')!=''){
    cat('\n eez & fao\n')
    x.shp=st_union(
        eez.simple[eez.simple$ISO_TER1%in%my.eez,],
        fao.simple[fao.simple$zone%in%my.fao,])
}else{
   if(paste(my.eez,collapse='')!=''){
        cat('\n eez only \n')
        x.shp=eez.simple[eez.simple$ISO_TER1%in%my.eez,]
   }else{
        if(paste(my.fao,collapse='')!=''){
            cat('\n fao only \n')
            x.shp=fao.simple[fao.simple$zone%in%my.fao,]
   }else{
            x.shp=eez.simple
    }}}

bbox.geo=suppressWarnings(
    sf::st_bbox(
        sf::st_buffer(
            x.shp,
            my.buff)
    ))

bbox.vec <- bbox.geo %>% as.vector()

df.bbox<-data.frame(
    wellknown::wkt2geojson(
        spocc::bbox2wkt(
            round(bbox.geo,2)))$geometry$coordinates)

colnames(df.bbox) <- c("lng", "lat")

df.bbox<<-df.bbox

list(
MAP=(
    leaflet()%>%
  leaflet::addProviderTiles(
    provider = 'Esri.OceanBasemap') %>%
  leaflet::addPolygons(
     data=eez.simple[eez.simple$ISO_TER1%in%my.eez,],
#     label=~UNION,
#        labelOptions=labelOptions(noHide = T, textOnly=T),                       
     color='red',
     weight=2, 
     fillOpacity = 0)%>%
  leaflet::addPolygons(
      data=fao.simple[fao.shp$zone%in%my.fao,],
      color='green', 
      weight=2, 
      fillOpacity = 0)%>%
  leaflet::addPolygons(
      lng=df.bbox$lng,
      lat=df.bbox$lat,
      color='blue', 
      weight=2, 
      fillOpacity=0)
    ),
BBOX=bbox.geo
)
}

In [62]:
set_bbox(
    my.fao=c('21'),
    my.eez=eez.TER('CAN',T))[['MAP']]


 eez & fao


although coordinates are longitude/latitude, st_union assumes that they are planar
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”dist is assumed to be in decimal degrees (arc_degrees).


HTML widgets cannot be represented in plain text (need html)

In [63]:
do.prep=function(
    limit=9,
    specs='Thunnus thynnus',
    provs='',
    eez='',
    fao='',
    my.buff=1.0,
    recenter=FALSE,
    debug=FALSE,
    useCache=FALSE){

start.time<<-Sys.time()

use.bbox<-set_bbox(my.eez=eez,my.fao=fao,my.buff=my.buff)[['BBOX']]
use.bbox<<-use.bbox
    
if(useCache & 'df.UNIQUE' %in% objects('.GlobalEnv')){
    
if (debug) cat('\nUsing Cache ...\n')
    
}else{

if(debug){cat('\nProcessing ...\n')}

cache.eez<<-eez  
cache.limit<<-limit    
cache.specs<<-specs

eez.shp=eez.shp[
    eez.shp$POL_TYPE=='Union EEZ and country',]    

## extract data    

if(debug){cat('bbox= ',use.bbox,'\n')}

df.raw=NULL
cat('\n use.bbox=',use.bbox,'\n')    
#use.bbox=c(-142.00257, 39.05314,  -46.70704,   87.42121)    
#cat('\n use.bbox=',use.bbox,'\n')    

for (species in specs){
    if(debug){cat(species,'...\n')}
    x=data.frame(
        spocc::occ2df(
            spocc::occ(query=species, 
                from=provs, 
                limit=limit, 
                has_coords = TRUE,
                geometry=use.bbox,
                throw_warnings = FALSE)
        )
    )
    if(length(x)!=0){
        df.raw=rbind(
            df.raw, 
            cbind(
                species=species,
                x)
        )
    }
}

df.raw<<-df.raw[!is.na(df.raw$latitude)&!is.na(df.raw$longitude),]

if(debug){
    cat('# df.raw counts ...\n')
    table(df.raw$species,df.raw$prov)%>%
        rbind(.,total=apply(.,2,sum,na.rm=TRUE))%>%
        cbind(.,total=apply(.,1,sum,na.rm=TRUE))%>%
        print(.,na.print='.') 
    cat('# elapsed time: ', Sys.time()-start.time,'\n')}

# scrub data ##########################################################
    
if(length(df.raw)==0){return(cat('\n*** NO DATA ***'))}
    
df<-df.raw

colnames(df)[colnames(df) == 'longitude'] <- 'lng'
colnames(df)[colnames(df) == 'latitude'] <- 'lat'

df$lat<-round(as.numeric(df$lat),12)
df$lng<-round(as.numeric(df$lng),12)

df<-dframe(df) %>%
    coord_impossible(lon='lng',lat='lat') %>%
    coord_incomplete(lon='lng',lat='lat') %>%
    coord_unlikely(lon='lng',lat='lat')
    
# fix dates ...
df<-df[!is.na(df$date),]
df<-df[as.character(df$date)>'1111-11-11',]

df<-df[df$date<=Sys.Date(),]
df.clean<<-df

if(debug){
    nrow(df.clean)
    range(df.clean$date)
    cat('# df.clean counts ...\n')
    table(df.clean$species,df.clean$prov)%>%
        cbind(.,ALL=apply(.,1,sum,na.rm=TRUE))%>%
        cbind(.,UNIQUE=table(count(df.clean,c('species','lng','lat','date'))$species))%>%
        cbind(.,DUPS=.[,'ALL']-.[,'UNIQUE'])%>%
        rbind(.,col.totals=apply(.,2,sum,na.rm=TRUE))%>%
        print(.,na.print='.') 
    head(df.clean)}
    
# eez, fao indexing #####################################################################

prov.pts<-count(df.clean,c('lng','lat'))[,1:2]
prov.pts$eez='UNK'
prov.pts$fao='99'

eez.pts<-sf::st_as_sf(
    prov.pts,
    coords = c('lng','lat'), 
    crs = sf::st_crs(eez.shp))

eez.intersect<-data.frame(
    sf::st_intersects(eez.pts, eez.shp))

for(i in 1:nrow(eez.intersect)){
    prov.pts$eez[eez.intersect$row.id[i]]<-eez.shp$ISO_TER1[eez.intersect$col.id[i]]
}

fao.pts<-sf::st_as_sf(
    prov.pts,coords = c('lng','lat'), 
    crs = sf::st_crs(fao.shp))

fao.nearest<-data.frame(
    sf::st_nearest_feature(fao.pts, fao.shp))

for(i in 1:nrow(fao.nearest)){
    prov.pts$fao[i]<-fao.shp$zone[fao.nearest[i,1]]    
}

df.clean<<-merge(df.clean,prov.pts,by=c('lng','lat'))
    
if(debug){
    cat('# df.clean counts by eez and fao ...')
    print(table(df.clean$eez,df.clean$fao))}
    
} 
###########################
#### end of prepartion ####
###########################

if(debug){
    cat('cache processing...\n')
}
    
df<-df.clean
if (recenter){ 
    df$lng <- ifelse(df$lng < -25, df$lng + 360, df$lng)
}

if(paste(specs,collapse='')!=''){
    if(nrow(df[df$species%in%specs,])>0){
        cat('specs= ',specs,'\n')
        df<-df[df$species%in%specs,]
    }else{
        cat('\n*** NO DATA FOR specs***\n')}
    }    

if(paste(eez,collapse='')!=''){
    if(nrow(df[df$eez%in%eez,])>0){
        cat('eez= ',eez,'\n')
        df<-df[df$eez%in%eez,]
    }else{
        cat('\n*** NO DATA FOR eez***\n')}
    }    

if(paste(specs,collapse='')!=''){
    if(nrow(df[df$species%in%specs,])>0){
        cat('specs= ',specs,'\n')
        df<-df[df$species%in%specs,]
    }else{
        cat('\n*** NO DATA FOR spec***\n')}
    }    

if(paste(fao,collapse='')!=''){
    if(nrow(df[df$fao%in%fao,])>0){
        cat('fao= ',fao,'\n')
        df<-df[df$fao%in%fao,]
    }else{cat('\n*** NO DATA FOR fao***\n')}
    }

if(paste(provs,collapse='')!=''){
    if(nrow(df[df$prov%in%provs,])>0){
        cat('provs= ',provs,'\n')
        df<-df[df$prov%in%provs,]
    }else{cat('\n*** NO DATA FOR provs***\n')}
    }    
    
df=df[!is.na(df$lat)&!is.na(df$lng),]
    
df.clean<-df
df.clean.debug<<-df
    
# create df.UNIQUE, ALL and DUPS ##########################################################

if(debug){
    nrow(df.clean)
    range(df.clean$date)
    cat('# df.clean counts ...\n')
    table(df.clean$species,df.clean$prov)%>%
        cbind(.,ALL=apply(.,1,sum,na.rm=TRUE))%>%
        cbind(.,UNIQUE=table(count(df.clean,c('species','lng','lat','date'))$species))%>%
        cbind(.,DUPS=.[,'ALL']-.[,'UNIQUE'])%>%
        rbind(.,col.totals=apply(.,2,sum,na.rm=TRUE))%>%
        print(.,na.print='.') 
    head(df.clean)}

df.UNIQUE<-count(df.clean,c('species','lng','lat','date','fao','eez'))
colnames(df.UNIQUE)[colnames(df.UNIQUE) == 'freq'] <- 'DUPS'
df.UNIQUE$id=seq.int(nrow(df.UNIQUE))
df.UNIQUE<<-df.UNIQUE
    
df.ALL<<-merge(df.clean,df.UNIQUE,by=c(c('species','lng','lat','date','fao','eez')))

df.DUPS<-count(df.ALL[df.ALL$DUP>1,],c('species','lng','lat','date','id','fao','eez'))
colnames(df.DUPS)[colnames(df.DUPS) == 'freq'] <- 'DUPS'
df.DUPS<-df.DUPS[,c('species','lng','lat','date','DUPS','id','fao','eez')]
df.DUPS<<-df.DUPS
    
if(debug){
    cat('\nALL=',nrow(df.ALL),'\n')
    cat('UNIQUE=',nrow(df.UNIQUE),'\n')
    cat('DUPS=',sum(df.DUPS$DUPS)-nrow(df.DUPS),'\n')
    cat('elapsed time: ', Sys.time()-start.time,'\n')}
    
results<<-list(
#    SUM=count(df.ALL,c('species','prov','eez','fao')),
    SUM=cat(
        '\nspecs=',cache.specs,
        '\nlimit=',cache.limit,
        '\nALL=',nrow(df.ALL),
        '\nUNIQUE=',nrow(df.UNIQUE),
        '\nDUPS=',sum(df.DUPS$DUPS)-nrow(df.DUPS)),
    TABLE=table(df.ALL$eez,df.ALL$fao),
    ALL=df.ALL,
    UNIQUE=df.UNIQUE,
    DUPS=df.DUPS,
    RAW=df.raw,
    CLEAN=df.clean)
results
}

In [64]:
my.eez=eez.TER('',T)
start.time=Sys.time()
#do.prep(specs='Semibalanus balanoides',      # acorn barnacle
#do.prep(specs='Caulerpa taxifolia',      # killer algae
#do.prep(specs='Pterois volitans',        # lion fish
#do.prep(specs='Thunnus thynnus',         # atlantic tuna
#do.prep(specs='Carcharodon carcharias',  # great white shark
#do.prep(specs='Balaenoptera musculus',   # blue whale
#do.prep(specs='Mirounga leonina',        # southern sea elephant
#do.prep(specs='Mirounga angustirostris', # northern sea elephant
#do.prep(specs='Odobenus rosmarus',       # walrus
#do.prep(specs='Salmo salar',             # atlantic salmon
seaweeds=c('Ascophyllum nodosum','Chondrus crispus','Laminaria digitata','Fucus vesiculosus',
'Rhodymenia palmata','Fucus distichus','Saccharina latissima','Codium fragile',
'Fucus spiralis','Fucus serratus','Palmaria palmata','Scytosiphon lomentaria','Furcellaria lumbricalis')
#

In [ ]:
do.prep(specs=seaweeds,
        eez='',
        fao='',
        limit=999,
        provs=c('obis','gbif','inat'),
        my.buff=0.0,
        debug=T)[['SUM']]
Sys.time()-start.time

dist is assumed to be in decimal degrees (arc_degrees).


In [214]:
qmap<-function(species='',my.fao='',my.eez='',buf=0,recenter=F){
pal <- 
   colorFactor(palette = c("red", "orange", "green", "cyan", "dodgerblue", "magenta"), 
               levels = c("ala", "bison", "gbif", "idigbio", "inat", "obis"))
df<<-do.prep(specs=species,eez=my.eez,fao=my.fao,provs='',useCache=T,debug=F,recenter=recenter)
leaflet()%>%
  leaflet::addProviderTiles(provider = 'Esri.OceanBasemap') %>%
  leaflet::addPolygons(
     data=eez.simple[eez.simple$ISO_TER1%in%my.eez,],
     color='red',
     weight=2, 
     fillOpacity = 0)%>%
  leaflet::addPolygons(
      data=fao.simple[fao.shp$zone%in%my.fao,],
      color='green', 
      weight=2, 
      fillOpacity = 0)%>%
  leaflet::addPolygons(
      lng=df.bbox$lng,
      lat=df.bbox$lat,
      color='blue', 
      weight=2, 
      fillOpacity=0)%>%
  leaflet::addCircleMarkers(data=count(df[['UNIQUE']],c('lat','lng')),color='grey',fillColor='grey',opacity=0.25,fillOpacity=0.25,radius=6)%>%
  leaflet::addCircleMarkers(data=df[['ALL']],color= ~pal(prov),radius=4,weight=2,fill=T)%>%
  leaflet::addCircleMarkers(data=df[['DUPS']],color='yellow',weight=1,radius=.75,opacity=1.0,fillOpacity=1.0,fill=T)%>%
  leaflet::fitBounds(min(df[['ALL']]$lng)-buf, min(df[['ALL']]$lat)-buf, max(df[['ALL']]$lng)+buf, max(df[['ALL']]$lat)+buf)
 }   

In [195]:
atlantic_ocean=c(21,31,41,27,34,47,37)
pacific_ocean=c(67,77,87,61,71,81)
indian_ocean=c(51,57)

In [217]:
#qmap(species='Codium fragile',my.eez=c('BHS','CAN','MEX',eez.TER('USA')),my.fao='',buf=0)
qmap(species='Codium fragile',my.eez='',my.fao='81',buf=0,recenter=T)



 fao only 


dist is assumed to be in decimal degrees (arc_degrees).


specs=  Codium fragile 
specs=  Codium fragile 
fao=  81 

specs= Ascophyllum nodosum Chondrus crispus Laminaria digitata Fucus vesiculosus Rhodymenia palmata Fucus distichus Saccharina latissima Codium fragile Fucus spiralis Fucus serratus Palmaria palmata Scytosiphon lomentaria Furcellaria lumbricalis 
limit= 999 
ALL= 307 
UNIQUE= 303 
DUPS= 4

Assuming "lng" and "lat" are longitude and latitude, respectively
Assuming "lng" and "lat" are longitude and latitude, respectively
Assuming "lng" and "lat" are longitude and latitude, respectively


HTML widgets cannot be represented in plain text (need html)

In [162]:
table(df[['ALL']]$species)


Codium fragile 
          2693 

In [232]:
my.eez=c('AUS','NZL')
my.fao=c('81')
leaflet()%>%
leaflet::addProviderTiles(provider = 'Esri.OceanBasemap') %>%
  leaflet::addPolygons(
     data=spTransform(eez.simple[eez.simple$ISO_TER1%in%my.eez,],3832),
     color='red',
     weight=2, 
     fillOpacity = 0)%>%
  leaflet::addPolygons(
      data=spTransform(fao.simple[fao.shp$zone%in%my.fao,],3832),
      color='green', 
      weight=2, 
      fillOpacity = 0)


ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function ‘spTransform’ for signature ‘"sf", "numeric"’


In [230]:
library(maptools)

Loading required package: sp
Checking rgeos availability: TRUE


In [235]:
xx<-spTransform(eez.simple,3832)

ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function ‘spTransform’ for signature ‘"sf", "numeric"’


In [234]:
library(rgdal)

rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.2, released 2021/03/05
Path to GDAL shared files: /opt/miniconda3/envs/conda_env/share/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 8.0.0, March 1st, 2021, [PJ_VERSION: 800]
Path to PROJ shared files: /opt/miniconda3/envs/conda_env/share/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
